In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pyuser_agent
import re
from datetime import datetime
from tqdm import tqdm
import unicodedata
from os import path
from os import listdir

In [2]:
posts_df=pd.read_csv('/home/slawomir/Portfolio/omdena_nlp_finance_chatbot/posts_bogleheads_to_scrape.csv')

In [3]:
posts_df

,topic_id,date,author,subject,replies,views,topic_link
0,393416,2022-12-28,AnnetteLouisan,What are you doing differently in 2023?,79,4359,https://www.bogleheads.org/forum/viewtopic.php...
1,387321,2022-10-03,Domadosolo,Trust or Don’t trust the keys to the kingdom i...,182,15855,https://www.bogleheads.org/forum/viewtopic.php...
2,393462,2022-12-28,monty,Filing for full SS,14,587,https://www.bogleheads.org/forum/viewtopic.php...
3,393350,2022-12-27,broncocountry25,"Kids, HCOL, & Savings",46,3354,https://www.bogleheads.org/forum/viewtopic.php...
4,391071,2022-11-26,ReardonSteel,Fidelity Holiday Offer - $150 for $50 deposit,303,36330,https://www.bogleheads.org/forum/viewtopic.php...
...,...,...,...,...,...,...,...
7809,366214,2021-12-31,yougotitdude,How to minimize financial risk for driving au ...,29,2526,https://www.bogleheads.org/forum/viewtopic.php...
7810,366251,2021-12-31,calimero,Paying quarterly Estimated taxes or simply inc...,7,686,https://www.bogleheads.org/forum/viewtopic.php...
7811,366233,2021-12-31,liz24,Help Me Understand My Real Estate Options,6,692,https://www.bogleheads.org/forum/viewtopic.php...
7812,366244,2021-12-31,bling,bank account recommendations for minors?,4,290,https://www.bogleheads.org/forum/viewtopic.php...


In [4]:
def url_sid_remover(url):
    url=re.sub('&sid=[a-f0-9]{32}', '', url)
    return url

In [5]:
def get_post_pages(url):
    url=url_sid_remover(url)
    pages_nums=[]
    pages_urls=[url]
    ua = pyuser_agent.UA()

    headers = {
      "User-Agent" : ua.random
    }
    try:
        res = requests.get(url,headers=headers).text
        soup=BeautifulSoup(res, 'html.parser')
        pages_div=soup.find_all('div', {'class':'pagination'})[0]
        if pages_div.find('ul')!=None:
            last_page=pages_div.find_all('a',{'class':'button'})[-2].text
            page_max_num=int(last_page)*50-50
            start=50
            while start<=page_max_num:
        
                pages_nums.append(start)
                start=start+50
            for p in pages_nums:
                url_page=f'{url}&start={p}'
                pages_urls.append(url_page)
    except AttributeError:
        pass
    except IndexError:
        pass
    return pages_urls


In [6]:
def get_reply_author(reply_cell):
    author=''
    try:
        author_cell=reply_cell.find('p', class_='author')
        author=author_cell.find('a', class_='username').text.strip()
    except AttributeError:
        pass
    return author

In [7]:
def get_reply_date(reply_cell):
    date=''
    try:
        date_cell=reply_cell.find('p', class_='author')
        date=date_cell.find('time')['datetime']
        date=datetime.strptime(date, '%Y-%m-%dT%H:%M:%S+00:00').date()
    except AttributeError:
        pass
    return date

In [8]:
def get_author_posts_count(reply_cell):
    replier_posts=''
    try:
        replier_cell=reply_cell.find('dl', class_='postprofile')
        replier_posts_cell=replier_cell.find('dd', class_='profile-posts')
        replier_posts_count=replier_posts_cell.find('a').text.strip()
        replier_posts=int(replier_posts_count)
    except AttributeError:
        pass
    return replier_posts

In [9]:
def get_reply_text(reply_cell):
    text=''
    try:
        text_cell=reply_cell.find('div', class_='postbody')
        text=text_cell.find('div', class_='content')
        try:
            blockquote_cell=text_cell.find_all('blockquote')
            if len(blockquote_cell)>0:
                for blockq in blockquote_cell:
                    blockq.extract()
            text=text.text
            text=unicodedata.normalize('NFKD', text).replace('\n',' ').strip()
        except AttributeError:
            pass
        text=text.text
        text=unicodedata.normalize('NFKD', text).replace('\n',' ').strip()    
    except AttributeError:
        pass

    return text
    

In [10]:
def get_post_text(url):
    text=''
    try:
        ua = pyuser_agent.UA()

        headers = {
      "User-Agent" : ua.random
        }
        content=requests.get(url, headers=headers).text
        soup = BeautifulSoup(content, "html.parser")
        
        post=soup.find_all('div', class_= re.compile('post has-profile bg'))[0]
        text_cell=post.find('div', class_='postbody')
        text=text_cell.find('div', class_='content')
        try:
            blockquote_cell=text_cell.find_all('blockquote')
            if len(blockquote_cell)>0:
                for blockq in blockquote_cell:
                    blockq.extract()
            text=text.text
            text=unicodedata.normalize('NFKD', text).replace('\n',' ').strip()
        except AttributeError:
            pass
        text=text.text
        text=unicodedata.normalize('NFKD', text).replace('\n',' ').strip()  
    except AttributeError:

        pass
    except IndexError:
        pass

    return text
    
    



In [11]:
def scrape_discussion(df,row):
    discussion_urls=get_post_pages(df['topic_link'].iloc[row])
    post_id=df['topic_id'].iloc[row]
    post_date=df['date'].iloc[row]
    post_title=df['subject'].iloc[row]
    post_author=df['author'].iloc[row]
    post_text=get_post_text(discussion_urls[0])
    
    discussion_data=[]
    for page in discussion_urls:
        
        
        
        date=None
        replier=None
        votes=None
        rank=None
        text=None
        ua = pyuser_agent.UA()

        headers = {
      "User-Agent" : ua.random
        }
        content=requests.get(page, headers=headers).text
        soup = BeautifulSoup(content, "html.parser")

        
        
        
        if page == discussion_urls[0]:
            
            
            replies=soup.find_all('div', class_= re.compile('post has-profile bg'))[1:]
            for reply in replies:
                date=get_reply_date(reply)
                
                replier=get_reply_author(reply)
                text=get_reply_text(reply)
                replier_posts_count=get_author_posts_count(reply)
                discussion_data.append((post_id, post_date, post_title, post_author, post_text, date, replier,text, replier_posts_count))
        else:
            
            
            replies=soup.find_all('div', class_= re.compile('post has-profile bg'))
            for reply in replies:
                date=get_reply_date(reply)
                replier=get_reply_author(reply)
                text=get_reply_text(reply)
                replier_posts_count=get_author_posts_count(reply)
                discussion_data.append((post_id, post_date, post_title, post_author, post_text, date, replier,text, replier_posts_count))


    posts_df=pd.DataFrame(discussion_data, columns=['post_id', 'post_date', 'post_title', 'post_author', 'post_text','reply_date', 'replier', 'reply_text','replier_posts_count'])
    posts_df.to_csv(f'/home/slawomir/Portfolio/omdena_nlp_finance_chatbot/boagleheads_data/{post_id}.csv')
    

In [12]:
posts_df=posts_df.iloc[4392:]

In [13]:
for i in tqdm(range(len(posts_df))):
    scrape_discussion(posts_df,i)


100%|██████████| 3422/3422 [3:11:55<00:00,  3.37s/it]  


In [2]:
all_files=listdir('/home/slawomir/Portfolio/omdena_nlp_finance_chatbot/boagleheads_data/')


main_paths=['/home/slawomir/Portfolio/omdena_nlp_finance_chatbot/boagleheads_data/']
all_paths_data=[]
for path_m in main_paths:
    
    all_files=listdir(path_m)
    for p in all_files:
        if '~lock' in p:
            continue
        file_path=str(path_m)+str(p)
        all_paths_data.append(file_path)

In [3]:
boagleheads_df=pd.concat((pd.read_csv(f,index_col=0) for f in all_paths_data), ignore_index=True )

In [4]:
boagleheads_df

,post_id,post_date,post_title,post_author,post_text,reply_date,replier,reply_text,replier_posts_count
0,390744,2022-11-20,CD beneficiary,brandy,I intend to tell a friend that he is to be pai...,2022-11-20,CAsage,Unless this is a person who you are committed ...,2549.0
1,381422,2022-07-08,Comparison shopping for insurance -- online vs...,28fe6,"I currently have car, homeowners, and umbrella...",2022-07-08,makingmistakes,I used to think shopping myself online made se...,401.0
2,381422,2022-07-08,Comparison shopping for insurance -- online vs...,28fe6,"I currently have car, homeowners, and umbrella...",2022-07-08,Broken Man 1999,I have tried online places where you enter you...,7871.0
3,378062,2022-05-20,"ACA insurance, married, does it matter who is ...",CloseEnough,Wondering if for a married couple purchasing h...,2022-05-20,JoeRetire,True. But not much in the way of administrativ...,14856.0
4,378062,2022-05-20,"ACA insurance, married, does it matter who is ...",CloseEnough,Wondering if for a married couple purchasing h...,2022-05-20,FrugalInvestor,In our case the older person (me) is listed as...,6059.0
...,...,...,...,...,...,...,...,...,...
138788,382352,2022-07-22,IRS penalty notice after amended return,wertdaman,My employer did not include ESPP income on my ...,2022-07-22,SuzBanyan,Is it clear why the penalty was assessed? If ...,1439.0
138789,382352,2022-07-22,IRS penalty notice after amended return,wertdaman,My employer did not include ESPP income on my ...,2022-07-22,crystalbank,Your refund shouldn't matter because it's alre...,518.0
138790,382352,2022-07-22,IRS penalty notice after amended return,wertdaman,My employer did not include ESPP income on my ...,2022-07-22,MarkNYC,"Having ""other taxes due past the deadline"", su...",2629.0
138791,382352,2022-07-22,IRS penalty notice after amended return,wertdaman,My employer did not include ESPP income on my ...,2022-07-22,UNCHEEL,"From my experiences with the IRS, you could ea...",119.0


In [5]:
boagleheads_df.to_csv('/home/slawomir/Portfolio/omdena_nlp_finance_chatbot/bogleheads_all_data_personal_finance.csv', index=False)